### DATA

In [1]:
import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm 


jazz_train = torch.rand(500,2,5,dtype=torch.float32)
rock_train = torch.rand(500,2,5,dtype=torch.float32)
jazz_train[:, 0, :] += 30
jazz_train[:, 1, :] += 60
rock_train[:, 0, :] += 80
rock_train[:, 1, :] += 10
y_genre_train = torch.cat(                                           
    (torch.zeros(500, dtype=torch.long),                          # 0 = jazz
     torch.ones(500, dtype=torch.long)),                          # 1 = rock
    dim=0
)    

X_music_train = torch.cat((jazz_train,rock_train),0)
train_music_dataset = TensorDataset(X_music_train,y_genre_train)
train_music_loader = DataLoader(train_music_dataset)

jazz_test = torch.rand(100,2,5,dtype=torch.float32)
rock_test = torch.rand(100,2,5,dtype=torch.float32)
jazz_test[:, 0, :] += 30
jazz_test[:, 1, :] += 60
rock_test[:, 0, :] += 80
rock_test[:, 1, :] += 10
test_music = torch.cat((jazz_test,rock_test),0)
y_genre_test = torch.cat(                                           
    (torch.zeros(100, dtype=torch.long),                          # 0 = jazz
     torch.ones(100, dtype=torch.long)),                          # 1 = rock
    dim=0
)    
x_music_test = torch.cat((jazz_test,rock_test),0)
test_music_dataset = TensorDataset(x_music_test,y_genre_test)
test_music_loader = DataLoader(test_music_dataset)
print(jazz_train,rock_train)

tensor([[[30.7325, 30.1777, 30.9958, 30.6967, 30.5440],
         [60.6846, 60.0415, 60.5159, 60.1413, 60.6475]],

        [[30.0474, 30.2120, 30.3131, 30.0668, 30.8934],
         [60.5839, 60.3474, 60.7822, 60.9362, 60.8814]],

        [[30.3536, 30.3188, 30.2022, 30.5008, 30.9970],
         [60.0454, 60.8237, 60.5710, 60.2148, 60.3626]],

        ...,

        [[30.8352, 30.8363, 30.9398, 30.4318, 30.0005],
         [60.3119, 60.6005, 60.7709, 60.9862, 60.3544]],

        [[30.6152, 30.9044, 30.4939, 30.5496, 30.9501],
         [60.8661, 60.1978, 60.3343, 60.0800, 60.3483]],

        [[30.4251, 30.9676, 30.4602, 30.4950, 30.1906],
         [60.2443, 60.9100, 60.4067, 60.0350, 60.7606]]]) tensor([[[80.6900, 80.6325, 80.4736, 80.8118, 80.9853],
         [10.6983, 10.4908, 10.2096, 10.9600, 10.2530]],

        [[80.2209, 80.5387, 80.7626, 80.1436, 80.7414],
         [10.5714, 10.2608, 10.6535, 10.2296, 10.7745]],

        [[80.6101, 80.2413, 80.4650, 80.5980, 80.6866],
         [10.9701,

### MODEL

In [2]:
model = nn.Sequential(nn.Conv1d(in_channels=2,out_channels=4,kernel_size=(2)),
                      nn.Conv1d(in_channels=4,out_channels=4,kernel_size=(2)),
                      nn.Flatten(),
                      nn.LazyLinear(out_features=2))#CrossEntropyLoss contrains sigmoid 


### Training

In [3]:
loss_fn = nn.CrossEntropyLoss()# compute the error 
optim_fn = optim.SGD(model.parameters(),lr=0.01) # Stochastic Gradient descent
for epoch in tqdm(range(200)):
    for xb,yb in train_music_loader: # 
        ypred = model(xb) # find the output of the model
        loss = loss_fn(ypred, yb)  #Out is the ML think is the answer, yb the real answer 
        optim_fn.zero_grad() # get ready for the slope
        loss.backward()# find where is it down the slope
        optim_fn.step()# Take a step down


100%|██████████| 200/200 [01:09<00:00,  2.87it/s]


### Predict

In [4]:
ypred = model(test_music)
argmax = torch.argmax(ypred,dim=1)
result = (argmax == y_genre_test)
acc = (result.sum()/len(y_genre_test)) * 100
print(f"{acc}%")

100.0%
